In [1]:
import numpy as np
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import tensorflow as tf

In [2]:
len_vcb = 366549
seed = 155
max_len = 150

In [3]:
with open('/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/utils/tokenizer_100k.json') as f: 
    data = json.load(f) 
    tok = tokenizer_from_json(data)

In [4]:
### definindo metricas customizadas

def recall(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # transformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_recall = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_pred[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_possibles = y_true[K.equal(y_true,K.constant(i))]/i
        possible_positives = K.sum(equals_possibles)

        recall = true_positives / (possible_positives + K.epsilon())
        macro_recall = macro_recall + recall

    macro_recall = macro_recall/(n_tags-1)  

    return macro_recall

def precision(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # trannsformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_precision = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_true[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_pred_possibles = y_pred[K.equal(y_pred,K.constant(i))]/i
        predicted_positives = K.sum(equals_pred_possibles)

        precision = true_positives / (predicted_positives + K.epsilon())
        macro_precision = macro_precision + precision

    macro_precision = macro_precision/(n_tags-1)

    return macro_precision

def macro_f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [5]:
custom_metrics = {"recall": recall,"precision": precision, "macro_f1": macro_f1}
model = tf.keras.models.load_model(
    '/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/SP_TOTAL_1EPOCH.h5',
    custom_objects=custom_metrics)

In [6]:
rio_de_janeiro_df = pd.read_json('/home/rafael/Documents/Faculdade/ICs/IC Direito/data/samples/sample_rio_de_janeiro_edital.json', lines=True)    

In [7]:
rio_de_janeiro_sample = rio_de_janeiro_df.sample(1, random_state=seed)['conteudo'].to_list()
rio_de_janeiro_sample

['Proc. 0000603-26.2010.8.19.0032 - SONIA MARIA DA SILVA RIBAS X ANA CRISTINA JANUÁRIO MONIZ DOS SANTOS Sentença: Ante ao exposto, JULGO EXTINTA A PRESENTE EXECUÇÃO de acordo com o art. 53, IV da Lei 9.099/95 podendo o exequente, contudo, socorrer-se do Juízo Cível competente, caso pretenda utilizar o procedimento disciplinado pelos artigos 754 e seguintes do Código de Processo Civil. Expeça-se Certidão de Crédito a favor do exequente, devendo a mesma ser instruída com os documentos pertinentes, bem como com o valor atualizado do débito. P.I. Façam as anotações e comunicações de estilo. Sem custas. Transitada em julgado, dê-se baixa e arquive-se.']

In [8]:
rio_de_janeiro_sample_tokenized = tok.texts_to_sequences(rio_de_janeiro_sample)
pd.DataFrame(rio_de_janeiro_sample_tokenized[0]).T

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,1342,1,17,159,15,479,2222,4454,181,61,...,132,1499,12,134,1385,8,1639,6,2479,8


In [9]:
rio_de_janeiro_sample_tokenized_padded = pad_sequences(maxlen=max_len,
                                    sequences=rio_de_janeiro_sample_tokenized,
                                    padding="post",
                                    truncating="post")
pd.DataFrame(rio_de_janeiro_sample_tokenized_padded)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,1342,1,17,159,15,479,2222,4454,181,61,...,0,0,0,0,0,0,0,0,0,0


In [10]:
predicted_rio_de_janeiro_sample_tokenized_padded = model(rio_de_janeiro_sample_tokenized_padded)

In [11]:
for i,p in enumerate(predicted_rio_de_janeiro_sample_tokenized_padded[0]):
    if (i < len(rio_de_janeiro_sample_tokenized[0])): 
        print(np.argmax(p),tok.index_word[rio_de_janeiro_sample_tokenized[0][i]])

1 Proc
1 UNK
1 26
1 2010
1 8
1 19
1 0032
1 SONIA
1 MARIA
1 DA
1 SILVA
1 RIBAS
1 X
1 ANA
1 CRISTINA
1 JANUÁRIO
1 MONIZ
1 DOS
1 SANTOS
1 Sentença
1 Ante
1 ao
1 exposto
1 JULGO
1 EXTINTA
1 A
1 PRESENTE
1 EXECUÇÃO
1 de
1 acordo
1 com
1 o
1 art
1 53
1 IV
1 da
1 Lei
1 9
1 099
1 95
1 podendo
1 o
1 exequente
1 contudo
1 socorrer
1 se
1 do
1 Juízo
1 Cível
1 competente
1 caso
1 pretenda
1 utilizar
1 o
1 procedimento
1 disciplinado
1 pelos
1 artigos
1 754
1 e
1 seguintes
1 do
1 Código
1 de
1 Processo
1 Civil
1 Expeça
1 se
1 Certidão
1 de
1 Crédito
1 a
1 favor
1 do
1 exequente
1 devendo
1 a
1 mesma
1 ser
1 instruída
1 com
1 os
1 documentos
1 pertinentes
1 bem
1 como
1 com
1 o
1 valor
1 atualizado
1 do
1 débito
1 P
1 I
1 Façam
1 as
1 anotações
1 e
1 comunicações
1 de
1 estilo
1 Sem
1 custas
1 Transitada
1 em
1 julgado
1 dê
1 se
1 baixa
1 e
1 arquive
1 se
